# Explorando Tokenización y Embeddings con Modelos de Lenguaje

💡 **NOTA**: Necesitaremos usar una GPU para ejecutar los ejemplos en este cuaderno.  
En Google Colab, andá a:  
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**


In [ ]:
%%capture
!pip install --upgrade pip
!pip uninstall -y transformers accelerate
!pip install transformers==4.41.2 accelerate==0.31.0 sentence-transformers==2.7.0 gensim==4.3.2 scikit-learn==1.5.0 numpy==1.26.4 scipy==1.13.1 pandas==2.0.3

## Descargando y Ejecutando un LLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

prompt = "Escribí un email disculpándote con María por el incidente en el asado del domingo. Explicá cómo sucedió."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=200
)
print(tokenizer.decode(generation_output[0]))

generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100
)
print(tokenizer.decode(generation_output[0]))


## Comparando Tokenizadores de LLMs Entrenados

In [ ]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

text = """
Español y MAYÚSCULAS
🧉 ñ
show_tokens False None elif == >= else: dos tabs:"    " Tres tabs: "       "
12.0*50=600
El mate está muy caliente
"""

show_tokens(text, "bert-base-uncased")
show_tokens(text, "bert-base-cased")
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")
show_tokens(text, "gpt2")


## Embeddings de Palabras Contextualizados desde un Modelo de Lenguaje

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

tokens = tokenizer('Hola mundo', return_tensors='pt')
output = model(**tokens)[0]

print("Forma del tensor de salida:")
print(output.shape)

print("\nTokens individuales:")
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))


In [ ]:
%%capture
!pip install --force-reinstall numpy==1.26.4 scipy==1.13.1

## Embeddings de Texto (Para Oraciones y Documentos Completos)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
vector = model.encode("¡La mejor película que vi!")

print("Dimensiones del vector:", vector.shape)
